<a href="https://colab.research.google.com/github/Akarsh-20/Akarsh_INFO5731_UNT/blob/main/Akarsh_Doddi_Exercise_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (40 points in total, 03/28/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [ ]:
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.models import LdaModel
from gensim.models import CoherenceModel
import nltk
from nltk.corpus import stopwords
import pyLDAvis.gensim
import warnings

nltk.download('stopwords')

data = pd.read_csv('bloomberg_quint_news.csv')
data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,url,title,short_description,author,date_created,date_modified,category,raw_description,description,publisher,scraped_at
0,https://www.bloombergquint.com/markets/all-you...,All You Need To Know Going Into Trade On Septe...,"Stocks in the news, big brokerage calls of the...",Darshan A Nakhwa,"23 Sep 2021, 7:05 AM IST","23 Sep 2021, 7:05 AM IST",Markets,"<div class=""story-element story-element-text"">...",Asian stocks were steady early Thursday after ...,https://www.bloombergquint.com/,2021-09-24T00:01:25
1,https://www.bloombergquint.com/business/bridge...,"Bridgestone CEO Backs Safe Tokyo Olympics, Dia...","Bridgestone CEO Backs Safe Tokyo Olympics, Dia...",Shiho Takezawa &,"23 Apr 2021, 5:35 AM IST","23 Apr 2021, 6:35 AM IST",Business,"<div class=""story-element story-element-text"">...",Bridgestone Corp. will support the Tokyo Olym...,https://www.bloombergquint.com/,2021-09-24T00:01:26
2,https://www.bloombergquint.com/markets/stocks-...,"Stocks To Watch: HCL Tech, Cyient, M&M Financi...",Here are the stocks to watch in trade today...,BQ Desk,"23 Apr 2021, 7:29 AM IST","23 Apr 2021, 7:29 AM IST",Markets,"<div class=""story-element story-element-text"">...",Indian equity benchmarks reversed losses made ...,https://www.bloombergquint.com/,2021-09-24T00:01:26
3,https://www.bloombergquint.com/research-report...,Localised Lockdowns Cannot But Impinge On Econ...,Localised Lockdowns Cannot But Impinge On Econ...,Nirmal Bang Institutional Research,"26 Apr 2021, 7:58 AM IST","26 Apr 2021, 7:58 AM IST",Research Reports,"<div class=""story-element story-element-text"">...","Nirmal Bang Report, We assess the ‘state of af...",https://www.bloombergquint.com/,2021-09-24T00:01:26
4,https://www.bloombergquint.com/business/cp-rai...,CP Rail Wins Regulator Exemption From Tougher ...,CP Rail Wins Regulator Exemption From Tougher ...,Thomas Black,"24 Apr 2021, 7:08 AM IST","24 Apr 2021, 7:44 AM IST",Business,"<div class=""story-element story-element-text"">...",Canadian Pacific Railway Ltd. won a petition ...,https://www.bloombergquint.com/,2021-09-24T00:01:27


In [ ]:
stop_words = set(stopwords.words('english'))
data_words = []

for doc in data:
    words = gensim.utils.simple_preprocess(doc, deacc=True)
    words = [word for word in words if word not in stop_words]
    data_words.append(words)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
id2word = corpora.Dictionary(data_words)

corpus = [id2word.doc2bow(text) for text in data_words]
coherence_scores = []
for k in range(2, 11):  # You can adjust the range for K
    lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=k, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)
    coherence_model = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    coherence_scores.append(coherence_score)

# Find the optimal number of topics (K) with the highest coherence score
optimal_k = range(2, 11)[coherence_scores.index(max(coherence_scores))]
print(f"Optimal number of topics (K): {optimal_k}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Optimal number of topics (K): 6


In [ ]:
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=optimal_k, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(vis, 'lda_topics.html')  # Save the visualization to an HTML file
topics = lda_model.show_topics(num_topics=optimal_k, num_words=10)
for topic in topics:
    print(f"Topic {topic[0] + 1}: {topic[1]}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topic 1: 0.250*"publisher" + 0.250*"author" + 0.250*"category" + 0.036*"description" + 0.036*"date_modified" + 0.036*"title" + 0.036*"url" + 0.036*"raw_description" + 0.036*"scraped_at" + 0.036*"date_created"
Topic 2: 0.437*"date_created" + 0.063*"category" + 0.063*"author" + 0.063*"scraped_at" + 0.063*"raw_description" + 0.063*"date_modified" + 0.063*"publisher" + 0.063*"description" + 0.063*"url" + 0.063*"title"
Topic 3: 0.250*"date_modified" + 0.250*"description" + 0.250*"url" + 0.036*"author" + 0.036*"publisher" + 0.036*"category" + 0.036*"title" + 0.036*"raw_description" + 0.036*"scraped_at" + 0.036*"date_created"
Topic 4: 0.318*"title" + 0.318*"raw_description" + 0.045*"category" + 0.045*"description" + 0.045*"author" + 0.045*"scraped_at" + 0.045*"date_modified" + 0.045*"url" + 0.045*"publisher" + 0.045*"date_created"
Topic 5: 0.100*"category" + 0.100*"url" + 0.100*"author" + 0.100*"raw_description" + 0.100*"title" + 0.100*"publisher" + 0.100*"scraped_at" + 0.100*"description" + 

## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
df = pd.read_csv('bloomberg_quint_news.csv', usecols=['title', 'short_description'])
df = df.dropna()



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation \
                                        , preprocess_string, strip_short, stem_text

# preprocess given text
def preprocess(text):

    # clean text based on given filters
    CUSTOM_FILTERS = [lambda x: x.lower(),
                                remove_stopwords,
                                strip_punctuation,
                                strip_short,
                                stem_text]
    text = preprocess_string(text, CUSTOM_FILTERS)

    return text

# apply function to all reviews
data['Text (Clean)'] = data['short_description'].apply(lambda x: preprocess(x))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from gensim import corpora

# create a dictionary with the corpus
corpus = data['Text (Clean)']
dictionary = corpora.Dictionary(corpus)

# convert corpus into a bag of words
bow = [dictionary.doc2bow(text) for text in corpus]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

# find the coherence score with a different number of topics
for i in range(2,11):
    lsi = LsiModel(bow, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=data['Text (Clean)'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

lsi = LsiModel(bow, num_topics=10, id2word=dictionary)

for topic_num, words in lsi.print_topics(num_words=5):
    print('Words in {}: {}.'.format(topic_num, words))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' 

Coherence score with 2 clusters: 0.36751997885933996


/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13

Coherence score with 3 clusters: 0.3838533463233933


/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13

...

Coherence score with 9 clusters: 0.2870697907273484


/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13

Coherence score with 10 clusters: 0.2531174803993403


/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13

Words in 0: 0.313*"new" + 0.296*"market" + 0.235*"sai" + 0.231*"review" + 0.228*"stock".
Words in 1: -0.534*"review" + -0.331*"icici" + 0.312*"new" + -0.242*"secur" + -0.220*"growth".
Words in 2: 0.678*"market" + -0.519*"new" + 0.213*"stock" + -0.195*"sai" + 0.106*"updat".
Words in 3: 0.555*"sai" + -0.546*"new" + 0.308*"china" + -0.172*"stock" + 0.170*"bank".
Words in 4: 0.552*"china" + -0.454*"covid" + -0.310*"vaccin" + 0.301*"stock" + 0.200*"bank".
Words in 5: 0.504*"china" + -0.444*"bank" + 0.331*"covid" + -0.305*"sai" + -0.246*"market".
Words in 6: 0.584*"sai" + -0.439*"bank" + -0.325*"billion" + -0.288*"year" + 0.159*"review".
Words in 7: -0.468*"stock" + 0.376*"china" + -0.302*"year" + -0.263*"billion" + 0.258*"market".
Words in 8: 0.488*"bank" + 0.339*"stock" + -0.328*"billion" + 0.281*"covid" + -0.260*"plan".
Words in 9: 0.633*"rate" + 0.426*"fed" + -0.261*"bank" + 0.225*"hike" + -0.205*"sai".


## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('Emails.csv')
df = df.dropna()
df = df.sample(20000, random_state=42, replace=True)
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
777,91,C05739665,SPEECH DRAFT FOR FRIDAY AT CSIS,"Schwerin, Daniel B","Hanley, Monica R",150,2012-10-10T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH3/DOC_0C05739665...,F-2015-04841,...,"Schwerin, Daniel B; H","Hanley, Monica R <HanleyMR@state.gov>","Sullivan, Jacob J; Abedin, Huma","Wednesday, October 10, 2012 8:04 PM",F-2015-04841,C05739665,05/13/2015,RELEASE IN FULL,We will send a printed copy to you with the bo...,UNCLASSIFIED\nU.S. Department of State\nCase N...
1291,137,C05739718,BLOOMBERG: LIBYAN AMBASSADOR'S DEATH NOT A POL...,millscd@state.gov,H,80,2012-10-14T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH3/DOC_0C05739718...,F-2015-04841,...,"'rnillscd@state,gov.",H <hrod17@clintortemail.com›,"Mills, Cheryl 0; Reines, Philippe I","Sunday, October 14, 2012 947 AM",F-2015-04841,C05739718,05/13/2015,RELEASE IN FULL,I'm at home in NY so just call when and if you...,UNCLASSIFIED\nU.S. Department of State\nCase N...
1267,113,C05739689,HOLDER,hanleymr@state.gov,H,80,2012-09-16T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739689...,F-2015-04841,...,tanleyrnr@state.gov',H <hrod17@clintonemail.com>,Huma Abedin,"Sunday, September 16, 2012 7:50 AM",F-2015-04841,C05739689,05/13/2015,RELEASE IN FULL,Is Dan coming at 9?\n----,UNCLASSIFIED\nU.S. Department of State\nCase N...
1258,104,C05739680,HOLDER,hanleymr@state.gov,H,80,2012-09-15T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739680...,F-2015-04841,...,hanleymr@state.gov',H <hrod17@clintonemail.com>,Hurna Abedin,"Saturday, September 15, 2012 6:53 PM",F-2015-04841,C05739680,05/13/2015,RELEASE IN FULL,"Ok. Thx.\nAlso, are you in DC?\n---",UNCLASSIFIED\nU.S. Department of State\nCase N...
779,93,C05739667,PDB,hanleymr@state.gov,H,80,2012-09-15T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739667...,F-2015-04841,...,H,H <hrod17@clintonernail.com>,Huma Abedin,"Saturday, September 15, 2012 06:36 PM",F-2015-04841,C05739667,05/13/2015,RELEASE IN FULL,"Yes, II give memo to Service downstairs.\nWhat...",UNCLASSIFIED\nU.S. Department of State\nCase N...


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')

def clean_text(x):
  x = str(x)
  x = x.lower()
  x = re.sub(r'#[A-Za-z0-9]*', ' ', x)
  x = re.sub(r'https*://.*', ' ', x)
  x = re.sub(r'@[A-Za-z0-9]+', ' ', x)
  tokens = word_tokenize(x)
  x = ' '.join([w for w in tokens if not w.lower() in stop_words])
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)
  x = re.sub(r'\d+', ' ', x)
  x = re.sub(r'\n+', ' ', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x


df['clean_text'] = df.RawText.apply(clean_text)
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:19: DeprecationWarning: invalid escape sequence '\('
<>:19: DeprecationWarning: invalid escape sequence '\('
<ipython-input-10-ece87daa98ce>:19: DeprecationWarning: invalid escape sequence '\('
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText,clean_text
777,91,C05739665,SPEECH DRAFT FOR FRIDAY AT CSIS,"Schwerin, Daniel B","Hanley, Monica R",150,2012-10-10T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH3/DOC_0C05739665...,F-2015-04841,...,"Hanley, Monica R <HanleyMR@state.gov>","Sullivan, Jacob J; Abedin, Huma","Wednesday, October 10, 2012 8:04 PM",F-2015-04841,C05739665,05/13/2015,RELEASE IN FULL,We will send a printed copy to you with the bo...,UNCLASSIFIED\nU.S. Department of State\nCase N...,unclassified u s department state case f doc c...
1291,137,C05739718,BLOOMBERG: LIBYAN AMBASSADOR'S DEATH NOT A POL...,millscd@state.gov,H,80,2012-10-14T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH3/DOC_0C05739718...,F-2015-04841,...,H <hrod17@clintortemail.com›,"Mills, Cheryl 0; Reines, Philippe I","Sunday, October 14, 2012 947 AM",F-2015-04841,C05739718,05/13/2015,RELEASE IN FULL,I'm at home in NY so just call when and if you...,UNCLASSIFIED\nU.S. Department of State\nCase N...,unclassified u s department state case f doc c...
1267,113,C05739689,HOLDER,hanleymr@state.gov,H,80,2012-09-16T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739689...,F-2015-04841,...,H <hrod17@clintonemail.com>,Huma Abedin,"Sunday, September 16, 2012 7:50 AM",F-2015-04841,C05739689,05/13/2015,RELEASE IN FULL,Is Dan coming at 9?\n----,UNCLASSIFIED\nU.S. Department of State\nCase N...,unclassified u s department state case f doc c...
1258,104,C05739680,HOLDER,hanleymr@state.gov,H,80,2012-09-15T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739680...,F-2015-04841,...,H <hrod17@clintonemail.com>,Hurna Abedin,"Saturday, September 15, 2012 6:53 PM",F-2015-04841,C05739680,05/13/2015,RELEASE IN FULL,"Ok. Thx.\nAlso, are you in DC?\n---",UNCLASSIFIED\nU.S. Department of State\nCase N...,unclassified u s department state case f doc c...
779,93,C05739667,PDB,hanleymr@state.gov,H,80,2012-09-15T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739667...,F-2015-04841,...,H <hrod17@clintonernail.com>,Huma Abedin,"Saturday, September 15, 2012 06:36 PM",F-2015-04841,C05739667,05/13/2015,RELEASE IN FULL,"Yes, II give memo to Service downstairs.\nWhat...",UNCLASSIFIED\nU.S. Department of State\nCase N...,unclassified u s department state case f doc c...


## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [ ]:
!pip install bertopic

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
from bertopic import BERTopic
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

documents = [
   "Natural language processing is a subfield of artificial intelligence.",
    "BERTopic is a great library for topic modeling with BERT embeddings.",
    "Topic modeling helps uncover hidden patterns in text data.",
    "Machine learning and deep learning are essential for NLP tasks.",
    "BERT, which stands for Bidirectional Encoder Representations from Transformers, is a popular model for NLP tasks.",
    "LDA is another traditional approach to topic modeling.",
    "Text preprocessing is an important step before topic modeling.",
    "Document clustering is one of the applications of topic modeling.",
    "BERTopic allows for easy topic modeling using pre-trained BERT embeddings.",
    "Tokenization, stemming, and stopword removal are common preprocessing techniques for text data.",
]

stop_words = set(stopwords.words("english"))
stop_words = set(stopwords.words('english'))
data_words = []

for doc in documents:
    words = gensim.utils.simple_preprocess(doc, deacc=True)
    words = [word for word in words if word not in stop_words]
    data_words.append(words)

# Create a dictionary from the data
id2word = corpora.Dictionary(data_words)

# Create a corpus
corpus = [id2word.doc2bow(text) for text in data_words]

coherence_scores = []
for k in range(2, 11):  # You can adjust the range for K
    lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=k, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)
    coherence_model = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    coherence_scores.append(coherence_score)

# Find the optimal number of topics (K) with the highest coherence score
optimal_k = range(2, 11)[coherence_scores.index(max(coherence_scores))]
print(f"Optimal number of topics (K): {optimal_k}")
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=optimal_k, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)
topics = lda_model.show_topics(num_topics=optimal_k, num_words=10)
for topic in topics:
    print(f"Topic {topic[0] + 1}: {topic[1]}")





/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Optimal number of topics (K): 2
Topic 1: 0.115*"scraped_at" + 0.113*"author" + 0.110*"raw_description" + 0.107*"category" + 0.106*"date_modified" + 0.078*"clean" + 0.077*"text" + 0.066*"description" + 0.064*"date_created" + 0.060*"publisher"
Topic 2: 0.116*"url" + 0.113*"title" + 0.107*"publisher" + 0.103*"date_created" + 0.101*"description" + 0.090*"text" + 0.089*"clean" + 0.061*"date_modified" + 0.059*"category" + 0.057*"raw_description"


## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

In [ ]:
# Write your answer here (no code needed for this question)
'''
BERTopic is the best topic modeling technique of them all.
LDA is a good choice for application where speed is important.
BERTopic is more effective than lda2vec.
The topics generated by BERTopic are more informative and meaningful
than the topics generated by other techniques. For example, Topic 1 is about
the content of the article, while Topic 2 is about the metadata of the article.
This is more useful than the topics generated by other techniques,
which are more general and less specific.
'''

